In [1]:
import os
import sys

sys.path.append("../")

from src.project import AWSProject
from src.modules import DynamoDB
from src.resources import LambdasConfig

In [2]:
db = DynamoDB(
    table_name="SensorData",
    read_capacity=20,
    write_capacity=20,
    hash_key_name="DeviceID",
    hash_key_type="S",
    sort_key_name="Timestamp",
    sort_key_type="N",
    isstream=True,
    stream_type="NEW_IMAGE",
)

project = AWSProject(
    dir=os.path.abspath("../terraform"),
    environement="dev",
    project_name="air-quality",
    aws_profil="eleve.isep.free",
    #secret_key = "siduisdfhbidsf",
    #access_key= "SDSDSDDSD",
    aws_region="ap-southeast-2",
)

project.set_database_module(db)

project.add_api(
    ressource_name="sensor",
    stage_name="v1",
    get=LambdasConfig(
        file_path="./lambdas_archives/point_query.zip",
        handler="point_query.lambda_handler",
        runtime="python3.9",
        memory_size=128,
        timeout=10,
    ),
    post=LambdasConfig(
        file_path="./lambdas_archives/point_put.zip",
        handler="point_put.lambda_handler",
        runtime="python3.9",
        memory_size=128,
        timeout=5,
    ),
)

project.enable_websocket_api(
    message=LambdasConfig(
        file_path="./lambdas_archives/websocket_message.zip",
        handler="websocket_message.handler",
        runtime="python3.9",
        memory_size=128,
        timeout=20,
    ),
    manager=LambdasConfig(
        file_path="./lambdas_archives/websocket_manager.zip",
        handler="websocket_manager.handler",
        runtime="python3.9",
        memory_size=128,
        timeout=5,
    ),
)

In [3]:
project.save()

In [4]:
!cd ../terraform && make

zip -j ./lambdas_archives/websocket_manager.zip ./functions/websocket_manager.py;  zip -j ./lambdas_archives/websocket_message.zip ./functions/websocket_message.py;  zip -j ./lambdas_archives/point_put.zip ./functions/point_put.py;  zip -j ./lambdas_archives/point_query.zip ./functions/point_query.py;
updating: websocket_manager.py (deflated 65%)
updating: websocket_message.py (deflated 65%)
updating: point_put.py (deflated 68%)
updating: point_query.py (deflated 62%)
terraform init
Initializing modules...

Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/aws from the dependency lock file
- Using previously-installed hashicorp/aws v4.32.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun

In [5]:
project.get_outputs()

{'account_id': '312155101165',
 'base_url': 'https://dn65pwpev8.execute-api.ap-southeast-2.amazonaws.com/SensorData-air-quality-devAPI-air-quality-dev-v1',
 'caller_arn': 'arn:aws:iam::312155101165:user/Admin-Prog',
 'caller_user': 'AIDAURLPE27W5EEL65KUF',
 'ingest_api_key': '<sensitive>',
 'websocket': 'wss://6usq37vqa4.execute-api.ap-southeast-2.amazonaws.com/WebsocketStreamAPI-v1',
 'key_name': 'APIKEY-SensorData-air-quality-devAPI-air-quality-dev',
 'key_secret': 'Ubfkbwdsbz5yb7d1T9z358NLwpYmevIF9cbUWhEJ'}